# 대규모 텐서플로 모델 훈련과 베포
* 웹 서비스로 만들면 각 시스템에서 2장에 소개한 간단한 REST API를 사용해 언제든지 모델에 쿼리를 던질 수 있다.
* 시간이 지나면 새로운 데이터에 모델을 정기적으로 훈련하고 엄데이트된 새 버전을 반영해야 한다.
* 제품이 성공적이라면 많은 QPS를 받기 시작하고 규모를 확장해야한다
* 서비스 규모를 확장하기 위한 좋은 방법은 구글 클라우드 AI 플랫폼 같은 클라우드 서비스로 TF 서빙을 사용하는 것.
* 훈련데이터가 많고 오래걸리면 모델의 장애물이 될것이다.
* 머신러닝은 아이디어가 좋은 성과를 낼지 미리 알기 어려우므로 최대한 많은 시도를 해보아야한다.
* GPU나TPU같은 하드웨어 가속기로 서버를 여러대 가지고 훈련하면 더 빨리 훈련할 수 있다.
---
## 텐서플로 모델 서빙
* 모델을 훈련했으면 쉽게 사용할 수 있다.
* 모델의 유일한 역할은 예측을 하는것이고 나머지 시스템은 쿼리를 한다.
* 이는 모델과 나머지 시스템을 분리하여 모델 버전을 쉽게 바꾸거나 필요에따라 규모를 늘리고 테스트를 할수 있다.
---
### 텐서플로 서빙 사용하기
* TF 섯빙은 C++로 자가성되고 많은 테스트를 거친 효율적인 모델 서버이다.
* TF서빙은 높은 부하를 처리할 수 있고 여러 모델을 서비스하며 모델 저장소에서 자동으로 최신 버전의 모델을 베포하는 등의 작업 수행 가능.
---
#### saved_model로 내보내기
* 텐서플로는 savedmodel 포맷으로 내보내기 위한 간편한 함수 제공

In [29]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")
np.random.seed(42)
tf.random.set_seed(42)
Project_Dir="/content/drive/MyDrive/데이터사이언스/hands-on-machinelearning/ch19"

In [30]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [31]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7012 - accuracy: 0.8241 - val_loss: 0.3715 - val_accuracy: 0.9024
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3536 - accuracy: 0.9020 - val_loss: 0.2990 - val_accuracy: 0.9144
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3036 - accuracy: 0.9145 - val_loss: 0.2651 - val_accuracy: 0.9272
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2736 - accuracy: 0.9231 - val_loss: 0.2436 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2509 - accuracy: 0.9296 - val_loss: 0.2257 - val_accuracy: 0.9364
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2322 - accuracy: 0.9350 - val_loss: 0.2121 - val_accuracy: 0.9396
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2161 - accuracy: 0.9401 - val_loss: 0.1970 - val_accuracy:

In [32]:
model_version="0001"
model_name="my_mnist_model"
model_path=os.path.join(Project_Dir,model_name,model_version)
tf.saved_model.save(model,model_path)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/데이터사이언스/hands-on-machinelearning/ch19/my_mnist_model/0001/assets


* 또는 모델의 save()메서드를 사용할 수 있다.(model.save(model_path))
* 파일의 확장자가 .h5가 아니면 HDF5 포멧 대신SavedModel포맷으로 저장한다.
* 이렇게하면 제품으로 배포했을 때 원래 형태 그대로 데이터를 주입할 수 있다.
* Saved_model은 모델의 한 버전을 나타낸다.
* 이 디렉터리에는 계산 그래프를 정의한 saved_model_pd파일과 변숫값을 담은 variables 디렉터리가 저장된다.
* 많은 개수의 가중치를 담은 모델의 경우 변숫값이 여러 개의 파일로 나눠 저장될 수 있다.
* 어휘 사전 파일,클래스 이름, 모델을 위한 샘플 데이터같은 부가적인 데이터가 들어 있는 assets 디렉터리를 포함한다.
* tf.saved_model.load()함수를 사용해 이를 로드할 수 있다.
* 변환 객체가 케라스 모델이 아니고 계산 그래프와 변숫값을 담은 SavedModel을 나타낸다. 이를 함수로 사용해서 예측을 만들 수 있다.

In [33]:
saved_model=tf.saved_model.load(model_path)
y_pred=saved_model(tf.constant(X_new))

* keras.models.load_model()로 케라스 모델로 직접 로드 할 수 있다.

In [34]:
model=keras.models.load_model(model_path)
y_pred1=model.predict(tf.constant(X_new,dtype=tf.float32))

In [35]:
# SavedModel을 검사할 수 있는 saved_model_cli 명령줄 도구를 제공한다.
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
serve


* SavedModel은 하나 이상의 메타그래프를 포함한다.
* 하나의 메타그래프는 하나의 계산 그래프와 함수 시그니처 정의이다.
* 각 메타그래프는 일련의 태그로 구분된다.
* saved_model_cli 명령을 사용해 예측을 만들 수 도 있다.

In [36]:
np.save(model_path+"my_mnist_tests.npy", X_new)
input_name = model.input_names[0]
input_name

'flatten_1_input'

#### 텐서플로 서빙 설치하기
* TF 서빙을 설치하는 방법은 여러가지이다.
* 도커 이미지를 사용하거나 시스템의 패키지 매니저를 사용하거나 소스에서 설치하는 등의 방법이다.


In [37]:
!docker pull tensorflow/serving

/bin/bash: docker: command not found


### GCP AI플랫폼에서 예측 서비스 만들기
---
### 예측 서비스 사용하기


In [43]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [41]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [45]:
tf.config.experimental.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [44]:
!nvidia-smi

Tue Aug  3 09:21:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    59W / 149W |    173MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------